In [132]:
import pandas as pd
import numpy as np

In [133]:
#set up base dataframe
df = pd.read_csv('/content/drive/MyDrive/ml-project/data/salaries.csv')
df.head()

,playerName,seasonStartYear,salary,inflationAdjSalary
0,Michael Jordan,1996,"$30,140,000","$52,258,566"
1,Horace Grant,1996,"$14,857,000","$25,759,971"
2,Reggie Miller,1996,"$11,250,000","$19,505,934"
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585"
4,Gary Payton,1996,"$10,212,000","$17,706,187"


In [134]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
df = df[df['seasonStartYear'] <= 2017]

In [136]:
#get player profile info
player_info = pd.read_csv('/content/drive/MyDrive/ml-project/data/player_info.csv')
player_info.head()

,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State"


In [137]:
#replace * and convert birthday to date time
player_info["playerName"] = player_info["playerName"].str.replace("*","")
player_info['birthDate'] = pd.to_datetime(player_info['birthDate']).dt.date
player_info.head()

<ipython-input-137-0c69eee07ba8>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  player_info["playerName"] = player_info["playerName"].str.replace("*","")


,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,1968-06-24,Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,1946-04-07,Iowa State
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,1947-04-16,UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,1969-03-09,LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,1974-11-03,"Michigan, San Jose State"


In [138]:
player_info.dtypes

playerName     object
From            int64
To              int64
Pos            object
Ht             object
Wt            float64
birthDate      object
Colleges       object
dtype: object

In [139]:
#figure out who is a duplicate name and start season
dup = player_info[player_info[['playerName','From']].duplicated(keep=False)]
dupNames = ["Bill Bradley","Tony Mitchell"]
dup

,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
469,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN
470,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton
3074,Tony Mitchell,2014,2014,F,6-6,216.0,1989-08-07,Alabama
3075,Tony Mitchell,2014,2014,F,6-8,235.0,1992-04-07,University of North Texas


In [140]:
#create columns
#TODO colleges, problem is some have multiple colleges
df["birthDate"] = np.nan
df["startYear"] = np.nan
df["position"] = np.nan
df["height"] = np.nan
df["weight"] = np.nan

for index, row in player_info.iterrows():
  name = row['playerName']
  if name not in dupNames:
    birth = player_info.loc[player_info['playerName'] == name]['birthDate'].values[0]
    df.loc[df["playerName"] == name, "birthDate"] = birth

    start = player_info.loc[player_info['playerName'] == name]['From'].values[0]
    df.loc[df["playerName"] == name, "startYear"] = start

    position = player_info.loc[player_info['playerName'] == name]['Pos'].values[0]
    df.loc[df["playerName"] == name, "position"] = position

    height = player_info.loc[player_info['playerName'] == name]['Ht'].values[0]
    df.loc[df["playerName"] == name, "height"] = height

    weight = player_info.loc[player_info['playerName'] == name]['Wt'].values[0]
    df.loc[df["playerName"] == name, "weight"] = weight
df.head()


,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0
1,Horace Grant,1996,"$14,857,000","$25,759,971",1965-07-04,1988.0,F-C,6-10,215.0
2,Reggie Miller,1996,"$11,250,000","$19,505,934",1965-08-24,1988.0,G-F,6-7,185.0
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585",1972-03-06,1993.0,C,7-1,325.0
4,Gary Payton,1996,"$10,212,000","$17,706,187",1968-07-23,1991.0,G,6-4,180.0


In [141]:
name = 'Michael Jordan'
rows = df.loc[df['playerName'] == name]
rows

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0
415,Michael Jordan,1997,"$33,140,000","$56,169,715",1963-02-17,1985.0,G-F,6-6,198.0
2573,Michael Jordan,2001,"$1,000,000","$1,526,382",1963-02-17,1985.0,G-F,6-6,198.0
3023,Michael Jordan,2002,"$1,030,000","$1,555,569",1963-02-17,1985.0,G-F,6-6,198.0


In [142]:
#get player box score
box = pd.read_csv('/content/drive/MyDrive/ml-project/data/boxscore.csv')
box.head()

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter
0,1,Sacramento Kings,Corliss Williamson,37:20,7,11,0,0,0,0,...,3,4,4,1,1,4,5,14,-2,1
1,1,Sacramento Kings,Mitch Richmond,32:00,6,12,1,4,1,1,...,5,5,3,1,2,3,1,14,-12,1
2,1,Sacramento Kings,Olden Polynice,31:34,0,4,0,0,1,4,...,5,7,3,1,0,4,3,1,-12,1
3,1,Sacramento Kings,Mahmoud Abdul-Rauf,29:27,7,13,1,2,2,2,...,2,2,5,1,1,2,2,17,-7,1
4,1,Sacramento Kings,Brian Grant,25:13,3,11,0,0,2,2,...,5,6,0,0,2,1,2,8,-7,1


In [143]:
#get player box score
games = pd.read_csv('/content/drive/MyDrive/ml-project/data/games.csv')
games.head()

,seasonStartYear,awayTeam,pointsAway,homeTeam,pointsHome,attendance,notes,startET,datetime,isRegular,game_id
0,1996,Sacramento Kings,85,Houston Rockets,96,16285.0,NaN,NaN,1996-11-01,1,1
1,1996,Los Angeles Clippers,97,Golden State Warriors,85,15593.0,NaN,NaN,1996-11-01,1,2
2,1996,Portland Trail Blazers,114,Vancouver Grizzlies,85,19193.0,NaN,NaN,1996-11-01,1,3
3,1996,Seattle SuperSonics,91,Utah Jazz,99,19911.0,NaN,NaN,1996-11-01,1,4
4,1996,New York Knicks,107,Toronto Raptors,99,28457.0,NaN,NaN,1996-11-01,1,5


In [144]:
year = 0
yearRange = []
years = []
for index, row in games.iterrows():
  y = row['seasonStartYear']
  if year != y:
    years.append(y)
    #print("year:",y)
    yearRange.append(row['game_id'])
    #print("start game id:",row['game_id'])
    year = y
yearRange.pop(0)
yearRange.append(30250)
years.pop()
yearRange.pop()
years.pop()
yearRange.pop()
print(years)
print(yearRange)

[1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
[1262, 2522, 3313, 4577, 5837, 7097, 8374, 9645, 10959, 12278, 13587, 14903, 16218, 17530, 18841, 19915, 21229, 22548, 23859, 25175, 26484, 27796]


In [145]:
seasons = pd.read_csv('/content/drive/MyDrive/ml-project/data/Seasons_Stats.csv')
seasons = seasons[seasons['Year'] >= 1996]
seasons.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
12293,12293,1996.0,Mahmoud Abdul-Rauf,PG,26.0,DEN,57.0,53.0,2029.0,18.6,...,0.930,26.0,112.0,138.0,389.0,64.0,3.0,115.0,117.0,1095.0
12294,12294,1996.0,Michael Adams,PG,33.0,CHH,21.0,3.0,329.0,16.1,...,0.743,5.0,17.0,22.0,67.0,21.0,4.0,25.0,25.0,114.0
12295,12295,1996.0,Rafael Addison,SF,31.0,CHH,53.0,0.0,516.0,9.6,...,0.773,25.0,65.0,90.0,30.0,9.0,9.0,27.0,74.0,171.0
12296,12296,1996.0,Cory Alexander,PG,22.0,SAS,60.0,0.0,560.0,8.7,...,0.640,9.0,33.0,42.0,121.0,27.0,2.0,68.0,94.0,168.0
12297,12297,1996.0,Jerome Allen,SG,23.0,MIN,41.0,0.0,362.0,7.8,...,0.722,5.0,20.0,25.0,49.0,21.0,5.0,34.0,42.0,108.0


In [146]:
seasons["Player"] = seasons["Player"].str.replace("*","")

<ipython-input-146-77a91d91d122>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  seasons["Player"] = seasons["Player"].str.replace("*","")


In [147]:
df["MP"] = np.nan
df["PTS"] = np.nan
df["Age"] = np.nan
df["team"] = np.nan
df["games"] = np.nan
df["games_started"] = np.nan
df["PER"] = np.nan #PER
df["FTr"] = np.nan#FTr
df["AST"] = np.nan#AST
df["STL"] = np.nan#STL
df["TRB"] = np.nan#TRB total rebounds
df["FT"] = np.nan#FT
df["3P"] = np.nan#3P
df["FG"] = np.nan#FG

In [148]:
for i in range(len(years)):
  y = yearRange[i]
  year = years[i]
  #print("year: ",year)
  #print("year range: ",prevYear,"-",y)
  players = seasons.loc[(seasons['Year'] == year)]
  names = players.Player.unique()
  #print(names)
  for n in names:
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'MP']= players.loc[players['Player'] == n, 'MP'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'PTS']= players.loc[players['Player'] == n, 'PTS'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'Age']= players.loc[players['Player'] == n, 'Age'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'team']= players.loc[players['Player'] == n, 'Tm'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'games']= players.loc[players['Player'] == n, 'G'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'games_started']= players.loc[players['Player'] == n, 'GS'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'PER']= players.loc[players['Player'] == n, 'PER'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'FTr']= players.loc[players['Player'] == n, 'FTr'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'AST']= players.loc[players['Player'] == n, 'AST'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'STL']= players.loc[players['Player'] == n, 'STL'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'TRB']= players.loc[players['Player'] == n, 'TRB'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'FT']= players.loc[players['Player'] == n, 'FT'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'3P']= players.loc[players['Player'] == n, '3P'].values[0]
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'FG']= players.loc[players['Player'] == n, 'FG'].values[0]

In [149]:
mj = df.loc[df['playerName'] == 'Michael Jordan']
mj

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight,MP,...,games,games_started,PER,FTr,AST,STL,TRB,FT,3P,FG
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0,3090.0,...,82.0,82.0,29.4,0.355,352.0,180.0,543.0,548.0,111.0,916.0
415,Michael Jordan,1997,"$33,140,000","$56,169,715",1963-02-17,1985.0,G-F,6-6,198.0,3106.0,...,82.0,82.0,27.8,0.304,352.0,140.0,482.0,480.0,111.0,920.0
2573,Michael Jordan,2001,"$1,000,000","$1,526,382",1963-02-17,1985.0,G-F,6-6,198.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3023,Michael Jordan,2002,"$1,030,000","$1,555,569",1963-02-17,1985.0,G-F,6-6,198.0,2093.0,...,60.0,53.0,20.7,0.252,310.0,85.0,339.0,263.0,10.0,551.0


In [150]:
df

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight,MP,...,games,games_started,PER,FTr,AST,STL,TRB,FT,3P,FG
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0,3090.0,...,82.0,82.0,29.4,0.355,352.0,180.0,543.0,548.0,111.0,916.0
1,Horace Grant,1996,"$14,857,000","$25,759,971",1965-07-04,1988.0,F-C,6-10,215.0,2286.0,...,63.0,62.0,17.9,0.306,170.0,62.0,580.0,152.0,1.0,347.0
2,Reggie Miller,1996,"$11,250,000","$19,505,934",1965-08-24,1988.0,G-F,6-7,185.0,2621.0,...,76.0,76.0,20.1,0.467,253.0,77.0,214.0,430.0,168.0,504.0
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585",1972-03-06,1993.0,C,7-1,325.0,1946.0,...,54.0,52.0,26.4,0.495,155.0,34.0,596.0,249.0,1.0,592.0
4,Gary Payton,1996,"$10,212,000","$17,706,187",1968-07-23,1991.0,G,6-4,180.0,3162.0,...,81.0,81.0,19.6,0.240,608.0,231.0,339.0,229.0,98.0,618.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,Marcus Thornton II,2017,"$46,079","$51,109",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10488,Jameel Warney,2017,"$46,079","$51,109",1994-01-31,2018.0,F,6-8,260.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10489,Aaron Jackson,2017,"$46,079","$51,109",1986-05-06,2018.0,G,6-4,185.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10490,Trey McKinney-Jones,2017,"$46,079","$51,109",1990-08-27,2018.0,G,6-5,214.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
df.isnull().sum()

playerName               0
seasonStartYear          0
salary                   0
inflationAdjSalary       0
birthDate              917
startYear              917
position               917
height                 917
weight                 917
MP                    2453
PTS                   2453
Age                   2453
team                  2453
games                 2453
games_started         2453
PER                   2454
FTr                   2464
AST                   2453
STL                   2453
TRB                   2453
FT                    2453
3P                    2453
FG                    2453
dtype: int64

In [152]:
df.count()

playerName            10492
seasonStartYear       10492
salary                10492
inflationAdjSalary    10492
birthDate              9575
startYear              9575
position               9575
height                 9575
weight                 9575
MP                     8039
PTS                    8039
Age                    8039
team                   8039
games                  8039
games_started          8039
PER                    8038
FTr                    8028
AST                    8039
STL                    8039
TRB                    8039
FT                     8039
3P                     8039
FG                     8039
dtype: int64

In [153]:
df.dtypes

playerName             object
seasonStartYear         int64
salary                 object
inflationAdjSalary     object
birthDate              object
startYear             float64
position               object
height                 object
weight                float64
MP                    float64
PTS                   float64
Age                   float64
team                   object
games                 float64
games_started         float64
PER                   float64
FTr                   float64
AST                   float64
STL                   float64
TRB                   float64
FT                    float64
3P                    float64
FG                    float64
dtype: object

In [154]:
df["position"] = df["position"].astype('category')
df["position_enc"] = df["position"].cat.codes
df["position"].value_counts()

G      3223
F      2650
C      1189
F-C     941
G-F     773
F-G     404
C-F     395
Name: position, dtype: int64

In [155]:
df["team"] = df["team"].astype('category')
df["team_enc"] = df["team"].cat.codes
df["team"].value_counts()

TOT    841
MIA    263
IND    260
UTA    257
SAS    256
DET    256
POR    255
LAL    254
BOS    253
MIN    251
CHI    250
LAC    249
DAL    249
HOU    247
ATL    240
GSW    240
NYK    237
TOR    237
WAS    230
ORL    230
MIL    229
SAC    228
DEN    227
CLE    226
PHI    221
PHO    212
MEM    180
NJN    177
SEA    160
CHA    105
OKC    100
NOH     97
CHH     73
VAN     64
BRK     55
NOP     53
CHO     33
WSB     23
NOK     21
Name: team, dtype: int64

In [156]:
df["team"].value_counts().count()

39

In [157]:
df[df["team"]=="TOT"]["team"]

25       TOT
36       TOT
39       TOT
50       TOT
52       TOT
        ... 
10380    TOT
10390    TOT
10398    TOT
10408    TOT
10454    TOT
Name: team, Length: 841, dtype: category
Categories (39, object): ['ATL', 'BOS', 'BRK', 'CHA', ..., 'UTA', 'VAN', 'WAS', 'WSB']

In [158]:
df.dtypes

playerName              object
seasonStartYear          int64
salary                  object
inflationAdjSalary      object
birthDate               object
startYear              float64
position              category
height                  object
weight                 float64
MP                     float64
PTS                    float64
Age                    float64
team                  category
games                  float64
games_started          float64
PER                    float64
FTr                    float64
AST                    float64
STL                    float64
TRB                    float64
FT                     float64
3P                     float64
FG                     float64
position_enc              int8
team_enc                  int8
dtype: object

In [159]:
df["height"] = df["height"].astype('category')
df["height_enc"] = df["height"].cat.codes
df["height"].value_counts()

6-9     1229
6-8      953
6-7      946
6-10     918
6-6      840
6-11     839
6-3      704
6-5      622
7-0      560
6-4      543
6-2      388
6-1      369
6-0      247
7-1      117
5-11      81
5-10      59
7-2       51
7-3       36
5-9       21
7-6       20
5-5       12
5-3        8
7-4        7
7-5        4
5-7        1
Name: height, dtype: int64

In [160]:
#all star cleaning
allStar = pd.read_csv('/content/drive/MyDrive/ml-project/data/allStar.csv')

In [161]:
df["all_star"] = 0
for i in range(len(years)):
  y = yearRange[i]
  year = years[i]
  players = allStar.loc[(allStar['Year'] == year)]
  names = players.Player.unique()
  for n in names:
    df.loc[(df['playerName'] == n) & (df['seasonStartYear'] == year),'all_star']= players.loc[players['Player'] == n, 'Selection Type'].values[0]

In [193]:
added = [["Kyrie Irving",2017,"starter"],["DeMar DeRozan",2017,"starter"],["Lebron James",2017,"starter"],["Jimmy Butler",2017,"starter"],["Giannis Antetokounmpo",2017,"starter"],
         ["Stephen Curry",2017,"starter"],["James Harden",2017,"starter"],["Kevin Durant",2017,"starter"],["Kawhi Leonard",2017,"starter"],["Anthony Davis",2017,"starter"],
         ["Isiah Thomas",2017,"reserve"],["John Wall",2017,"reserve"],["Kevin Love",2017,"reserve"],["Carmelo Anthony",2017,"replacement"],["Kyle Lowry",2017,"reserve"],
         ["Paul George",2017,"reserve"],["Kemba Walker",2017,'reserve'],["Paul Milsap",2017,"reserve"], ["Russell Westbrook",2017,"reserve"],["Klay Thompson",2017,"reserve"],
         ["Draymond Green",2017,"reserve"],["DeMarcus Cousins",2017,"reserve"],["Marc Gasol",2017,"reserve"],["DeAndre Jordan",2017,"reserve"],["Gordon Hayward",2017,"reserve"],
         ["Gary Payton",1998,"starter"],["Kobe Bryant",1998,"starter"],["Kevin Garnett",1998,"starter"],["Karl Malone",1998,"starter"],["Shaquille O'Neal",1998,"starter"],
         ["Penny Hardaway",1998,"starter"],["Michael Jordan",1998,"starter"],["Grant Hill",1998,"starter"],["Shawn Kemp",1998,"starter"],["Dikembe Mutombo",1998,"starter"],
         ["Tim Hardaway",1998,"reserve"],["Reggie Miller",1998,"reserve"],["Steve Smith",1998,"reserve"],["Glen Rice",1998,"reserve"],["Antoine Walker",1998,"reserve"],
         ["Jayson Williams",1998,"reserve"],["Rik Smits",1998,"reserve"],["Jason Kidd",1998,"reserve"],["Eddie Jones",1998,"reserve"],["Nick Van Exel",1998,"reserve"],
         ["Mitch Richmond",1998,"reserve"],["Tim Duncan",1998,"reserve"],["Vin Baker",1998,"reserve"],["Tim Hardaway",1998,"reserve"],["David Robinson",1998,"reserve"],
         ["Penny Hardaway",1997,"starter"],["Michael Jordan",1997,"starter"],["Scottie Pippen",1997,"starter"],["Grant Hill",1997,"starter"],["Patrick Ewing",1997,"starter"],
         ["Gary Payton",1997,"starter"],["John Stockton",1997,"starter"],["Charles Barkley",1997,"starter"],["Shawn Kemp",1997,"starter"],["Hakeem Olajuwon",1997,"starter"],
         ["Vin Baker",1997,"reserve"],["Terrell Brandon",1997,"reserve"],["Joe Dumars",1997,"replacement"],["Tim Hardaway",1997,"reserve"],["Christian Laettner",1997,"reserve"],
         ["Alonzo Mourning",1997,"reserve"],["Dikembe Mutombo",1997,"reserve"],["Glen Rice",1997,"reserve"],["Chris Webber",1997,"replacement"],["Clyde Drexler",1997,"reserve"],
         ["Kevin Garnett",1997,"replacement"],["Chris Gatling",1997,"replacement"],["Tom Gugliotta",1997,"reserve"],["Eddie Jones",1997,"reserve"],["Karl Malone",1997,"reserve"],
         ["Shaquille O'Neal",1997,"reserve"],["Mitch Richmond",1997,"reserve"],["Detlef Schrempf",1997,"replacement"],["Letrell Sprewell",1997,"reserve"],
         ["Penny Hardaway",1996,"starter"],["Michael Jordan",1996,"starter"],["Grant Hill",1996,"starter"],["Scottie Pippen",1996,"starter"],["Shaquille O'Neal",1996,"starter"],
         ["Jason Kidd",1996,"starter"],["Clyde Drexler",1996,"starter"],["Charles Barkley",1996,"starter"],["Shawn Kemp",1996,"starter"],["Hakeem Olajuwon",1996,"starter"],
         ["Vin Baker",1996,"reserve"],["Terrell Brandon",1996,"reserve"],["Patrick Ewing",1996,"reserve"],["Juwan Howard",1996,"reserve"],["Reggie Miller",1996,"reserve"],
         ["Alonzo Mourning",1996,"reserve"],["Glen Rice",1996,"reserve"],["Sean Elliot",1996,"reserve"],["Karl Malone",1996,"reserve"],["Dikembe Mutombo",1996,"reserve"],
          ["Gary Payton",1996,"reserve"],["Mitch Richmond",1996,"reserve"],["David Robinson",1996,"reserve"],["John Stockton",1996,"reserve"]
         ]
status = pd.DataFrame(added, columns=['name', 'year','status'])
for index, row in status.iterrows():
  name = row['name']
  year = row['year']
  status = row['status']
  df.loc[(df['playerName'] == name) & (df['seasonStartYear'] == year),'all_star']= status

In [194]:
df.head()

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight,MP,...,STL,TRB,FT,3P,FG,position_enc,team_enc,height_enc,all_star,all_star_enc
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0,3090.0,...,180.0,543.0,548.0,111.0,916.0,6,5,14,starter,0
1,Horace Grant,1996,"$14,857,000","$25,759,971",1965-07-04,1988.0,F-C,6-10,215.0,2286.0,...,62.0,580.0,152.0,1.0,347.0,3,26,8,0,0
2,Reggie Miller,1996,"$11,250,000","$19,505,934",1965-08-24,1988.0,G-F,6-7,185.0,2621.0,...,77.0,214.0,430.0,168.0,504.0,6,13,15,reserve,0
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585",1972-03-06,1993.0,C,7-1,325.0,1946.0,...,34.0,596.0,249.0,1.0,592.0,0,26,19,starter,0
4,Gary Payton,1996,"$10,212,000","$17,706,187",1968-07-23,1991.0,G,6-4,180.0,3162.0,...,231.0,339.0,229.0,98.0,618.0,5,32,12,reserve,0


In [195]:
df["all_star"].value_counts()

0              9970
reserve         285
starter         202
replacement      35
Name: all_star, dtype: int64

In [196]:
df['all_star'] = df['all_star'].replace(['Eastern All-Star Fan Vote Selection','Western All-Star Fan Vote Selection','Eastern All-Star Coaches Selection',
                                         'Western All-Star Coaches Selection','Eastern All-Star Replacement Selection','Western All-Star Replacement Selection'], 
                                        ['starter','starter','reserve','reserve','replacement','replacement'])
df["all_star"].value_counts()

0              9970
reserve         285
starter         202
replacement      35
Name: all_star, dtype: int64

In [197]:
df["all_star_enc"] = df['all_star'].copy()
df['all_star_enc'] = df['all_star_enc'].replace(['starter','reserve','replacement'], 
                                        [3,2,1])

In [198]:
df.loc[(df['all_star'] == "reserve")]

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight,MP,...,STL,TRB,FT,3P,FG,position_enc,team_enc,height_enc,all_star,all_star_enc
2,Reggie Miller,1996,"$11,250,000","$19,505,934",1965-08-24,1988.0,G-F,6-7,185.0,2621.0,...,77.0,214.0,430.0,168.0,504.0,6,13,15,reserve,2
4,Gary Payton,1996,"$10,212,000","$17,706,187",1968-07-23,1991.0,G,6-4,180.0,3162.0,...,231.0,339.0,229.0,98.0,618.0,5,32,12,reserve,2
5,David Robinson,1996,"$9,952,000","$17,255,383",1965-08-06,1990.0,C,7-1,235.0,3019.0,...,111.0,1000.0,626.0,3.0,711.0,0,31,19,reserve,2
6,Juwan Howard,1996,"$9,750,000","$16,905,143",1973-02-07,1995.0,F,6-9,240.0,3294.0,...,67.0,660.0,319.0,4.0,733.0,2,38,17,reserve,2
8,Alonzo Mourning,1996,"$9,380,000","$16,263,615",1970-02-08,1993.0,C,6-10,240.0,2671.0,...,70.0,727.0,488.0,9.0,563.0,0,17,8,reserve,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9954,DeMarcus Cousins,2017,"$18,063,850","$20,035,826",1990-08-13,2011.0,C-F,6-10,270.0,2465.0,...,100.0,797.0,517.0,131.0,647.0,1,34,8,reserve,2
9955,John Wall,2017,"$18,063,850","$20,035,826",1990-09-06,2011.0,G,6-3,210.0,2836.0,...,157.0,326.0,422.0,89.0,647.0,5,37,11,reserve,2
9959,Klay Thompson,2017,"$17,826,150","$19,772,177",1990-02-08,2012.0,G-F,6-6,215.0,2649.0,...,66.0,285.0,186.0,268.0,644.0,6,11,14,reserve,2
9971,Draymond Green,2017,"$16,400,000","$18,190,338",1990-03-04,2013.0,F,6-6,230.0,2471.0,...,154.0,599.0,151.0,81.0,272.0,2,11,14,reserve,2


In [199]:
df.head()

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight,MP,...,STL,TRB,FT,3P,FG,position_enc,team_enc,height_enc,all_star,all_star_enc
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0,3090.0,...,180.0,543.0,548.0,111.0,916.0,6,5,14,starter,3
1,Horace Grant,1996,"$14,857,000","$25,759,971",1965-07-04,1988.0,F-C,6-10,215.0,2286.0,...,62.0,580.0,152.0,1.0,347.0,3,26,8,0,0
2,Reggie Miller,1996,"$11,250,000","$19,505,934",1965-08-24,1988.0,G-F,6-7,185.0,2621.0,...,77.0,214.0,430.0,168.0,504.0,6,13,15,reserve,2
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585",1972-03-06,1993.0,C,7-1,325.0,1946.0,...,34.0,596.0,249.0,1.0,592.0,0,26,19,starter,3
4,Gary Payton,1996,"$10,212,000","$17,706,187",1968-07-23,1991.0,G,6-4,180.0,3162.0,...,231.0,339.0,229.0,98.0,618.0,5,32,12,reserve,2


In [200]:
df.to_csv('/content/drive/MyDrive/ml-project/data/cleaned.csv')